# WEFE Site Analyst Tool
## implemented in Jupyter Notebook and applied for a Case Study in Portugal

This Jupyter Notebook collects environmental and socioeconomic data for the planning of integrated water, energy, food, and environment systems from open servers.

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import os
#import folium
#import geojson
import geemap
import geemap.foliumap as gfolium
import json
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON

from shapely.geometry import Polygon, shape

# import files from source
os.chdir("../../src/")
import era5
import ee_layer
os.chdir("../examples/Portugal")



In [2]:
# initiate earth engine
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AWtgzh6e1QY5ZB_S7mWUtWFHf7Gg8_97FGZEVyY8B4nCSuR75oFXBa9W_tA

Successfully saved authorization token.


In [4]:
## CASE STUDY LOCATION and PROJECT AREA
name = 'permontanha'

# coordinates of center project location (e.g. house of main demand)
lat = 37.134986
lon = -8.853894

# create GeoJSON for the specific point
center = {
  "type": "Point",
  "coordinates": [lon, lat]
}

# Calculation of project area
# Convert the GeoJSON to a Shapely polygon
# polygon_shapely = shape(polygon_json)

# Calculate the area of the polygon
# area = polygon.area

# print(area)

# Shape of target area

file_path = os.path.abspath('pp_geometry.geojson')
with open(file_path) as f:
    polygon_json = json.load(f)
    
print(polygon_json)

# Create ipiLEAFlet layer from the GeoJSONs    
json_shape_layer = GeoJSON(
    data=polygon_json,
    name='target area',
    hover_style={'fillColor': 'red', 'fillOpacity': 0.1},
)
json_center =GeoJSON(data=center,
                    name='center'
)

# visualization as geemap
map = geemap.Map(center=[lat, lon], zoom=17)
# map.add_basemap('TERRAIN')
map.add_basemap('SATELLITE')
map.add_layer(json_shape_layer)
map.add_layer(json_center)

map


{'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}, 'features': [{'type': 'Feature', 'properties': {'id': 1}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-8.854397164323958, 37.13384347121716], [-8.854705519411379, 37.1338743067259], [-8.854940640165536, 37.13403233870821], [-8.85511408990221, 37.13407859197132], [-8.85546484381415, 37.13419037069051], [-8.855279830761699, 37.13458352342697], [-8.855040855568948, 37.1348841696372], [-8.85476333599027, 37.135134708145735], [-8.853217706114576, 37.13589017810991], [-8.853202288360205, 37.135963412443175], [-8.852123045554233, 37.13587090591695], [-8.852350457431205, 37.135616512969825], [-8.852786008992187, 37.13519252472462], [-8.852839971132486, 37.13514241702292], [-8.853040401939309, 37.134880315198615], [-8.853352611465322, 37.1346721755146], [-8.853637839921186, 37.13443705476045], [-8.85399244827172, 37.134132554111616], [-8.854397164323958, 37.13384347121716]]]}}]}

Map(center=[37.134986, -8.853894], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

In [5]:
## TERRAIN MAP
# Define the location for which you want to create the elevation map (given abo)

center = ee.Geometry.Point(lon, lat)
buffer = center.buffer(3000)

# geemap.show_youtube('OlNlqfj4uHo')

# Create a geemap map centered on the location
terrain_map = gfolium.Map(center=[lat, lon], zoom=17)

# load shape of project area
file_path = os.path.abspath('pp_geometry.geojson')
with open(file_path) as f:
    json_shape = json.load(f)

ee_shape = geojson_to_ee(json_shape)
ee_data = geojson_to_ee(json_data)
Map.addLayer(ee_data, {}, "US States EE")


# polygon_shp = 'pp_geometry.shp'
# polygon = geemap.shp_to_ee(polygon_shp)
# map.addLayer(polygon, {}, 'target_area')

# Use the 'SRTM 30m Digital Elevation Database' dataset to get the elevation
elevation = ee.Image('USGS/SRTMGL1_003').clip(buffer);
slope = ee.Terrain.slope(elevation);
# Calculate aspect. Units are degrees where 0=N, 90=E, 180=S, 270=W.
aspect = ee.Terrain.aspect(elevation);
terrain = ee.Terrain.products(elevation);

vis_params = {
    'min': 0,
    'max': 200,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

vis_params_slope = {
    'min': 0,
    'max': 20,
    'palette': ['FFFFFF', '000000'],
}

# map.add_ee_layer(elevation, {'min': 70, 'max': 150, 'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}, 'elevation')
terrain_map.addLayer(elevation, vis_params, 'elevation')
terrain_map.addLayer(slope,{'min': 0, 'max': 20, 'palette': ['FFFFFF', '000000']}, 'slope')
terrain_map.addLayer(aspect,{'min': 0, 'max': 360, 'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}, 'aspect')
terrain_map.addLayer(terrain.select('hillshade'), {min: 0, max: 255}, 'Hillshade')
terrain_map.add_basemap('SATELLITE')
terrain_map.addLayer(ee_shape,{'color': 'blue', 'weight': 1, 'opacity': 0.4, 'fillColor' : 'red',  'fillOpacity' : 0}, "shape")
terrain_map.addLayer(center,{'color': 'cyan', 'weight': 1, 'opacity': 0.7}, "center")

colors = vis_params['palette']
vmin = vis_params['min']
vmax = vis_params['max']

terrain_map.add_colorbar_branca(vis_params = vis_params, colors=colors, vmin=vmin, vmax=vmax, layer_name="elevation")
terrain_map.add_colorbar_branca(vis_params = vis_params_slope, colors = vis_params_slope['palette'], vmin= vis_params_slope['min'], vmax= vis_params_slope['max'], layer_name="slope")

# Display the map
terrain_map


In [ ]:
## LAND USE, NDVI and POPULATION


In [ ]:
## Soil Properties

#soil organic carbon
soc_mean = ee.Image("projects/soilgrids-isric/soc_mean")
print(type(soc_mean))

# Soil salinity
# Soil depth
# soil bulk density
bdod_mean = ee.Image("projects/soilgrids-isric/bdod_mean")
# Soil Cation Exchange Capacity
cec_mean = ee.Image("projects/soilgrids-isric/cec_mean")
# Soil moisture
#nitrogen
nitrogen_mean = ee.Image("projects/soilgrids-isric/nitrogen_mean")
# Soil pH (in H2O)
phh2o_mean = ee.Image("projects/soilgrids-isric/phh2o_mean")
# soil texture
clay_mean = ee.Image("projects/soilgrids-isric/clay_mean")
silt_mean = ee.Image("projects/soilgrids-isric/silt_mean")
sand_mean = ee.Image("projects/soilgrids-isric/sand_mean")

location = {
  "type": "Point",
  "coordinates": [lon, lat]
}

# Extract the soil properties for the location
soc_mean = soc_mean.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=ee.Geometry.Point(location["coordinates"]),
    scale=250
).getInfo()

bdod_mean = bdod_mean.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=ee.Geometry.Point(location["coordinates"]),
    scale=250
).getInfo()

cec_mean = cec_mean.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=ee.Geometry.Point(location["coordinates"]),
    scale=250
).getInfo()

nitrogen_mean = nitrogen_mean.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=ee.Geometry.Point(location["coordinates"]),
    scale=250
).getInfo()

phh2o_mean = phh2o_mean.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=ee.Geometry.Point(location["coordinates"]),
    scale=250
).getInfo()

clay_mean = clay_mean.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=ee.Geometry.Point(location["coordinates"]),
    scale=250
).getInfo()

silt_mean = silt_mean.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=ee.Geometry.Point(location["coordinates"]),
    scale=250
).getInfo()
index_list = ['0-5cm', '100-200cm', '15-30cm', '30-60cm', '5-15cm', '60-100cm']

sand_mean = sand_mean.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=ee.Geometry.Point(location["coordinates"]),
    scale=250
).getInfo()

# Convert the soil properties to a pandas dataframe
soc_mean_df = pd.DataFrame.from_dict(soc_mean, orient='index', columns=["soil organic carbon [dg/kg]"])
soc_mean_df['soil_depth'] = index_list
soc_mean_df = soc_mean_df.set_index(keys='soil_depth')

bdod_mean_df = pd.DataFrame.from_dict(bdod_mean, orient='index', columns=["bulk density [cg/cm³]"])
bdod_mean_df['soil_depth'] = index_list
bdod_mean_df = bdod_mean_df.set_index(keys='soil_depth')

cec_mean_df  = pd.DataFrame.from_dict(cec_mean, orient='index', columns=["cation exchange capacity at pH 7 [mmol(c)/kg]"])
cec_mean_df['soil_depth'] = index_list
cec_mean_df = cec_mean_df.set_index(keys='soil_depth')

nitrogen_mean_df = pd.DataFrame.from_dict(nitrogen_mean, orient='index', columns=["nitrogen content [cg/kg]"])
nitrogen_mean_df['soil_depth'] = index_list
nitrogen_mean_df = nitrogen_mean_df.set_index(keys='soil_depth')

phh2o_mean_df = pd.DataFrame.from_dict(phh2o_mean, orient='index', columns=["pH of soil water [pH*10]"])
phh2o_mean_df['soil_depth'] = index_list
phh2o_mean_df = phh2o_mean_df.set_index(keys='soil_depth')

clay_mean_df = pd.DataFrame.from_dict(clay_mean, orient='index', columns=["clay content [g/kg]"])
clay_mean_df['soil_depth'] = index_list
clay_mean_df = clay_mean_df.set_index(keys='soil_depth')

silt_mean_df = pd.DataFrame.from_dict(silt_mean, orient='index', columns=["silt content [g/kg]"])
silt_mean_df['soil_depth'] = index_list
silt_mean_df = silt_mean_df.set_index(keys='soil_depth')

sand_mean_df = pd.DataFrame.from_dict(clay_mean, orient='index', columns=["sand content [g/kg]"])
sand_mean_df['soil_depth'] = index_list
sand_mean_df = sand_mean_df.set_index(keys='soil_depth')

soil_properties_df = pd.concat((soc_mean_df, bdod_mean_df, cec_mean_df, nitrogen_mean_df, phh2o_mean_df, clay_mean_df, silt_mean_df, sand_mean_df), axis=1)
print(soil_properties_df)


def soil_texture(row):
    clay =row["clay content [g/kg]"]
    silt =row["silt content [g/kg]"]
    sand =row["sand content [g/kg]"]
    
    total = clay + silt + sand
    clay_percent = clay / total * 100
    silt_percent = silt / total * 100
    sand_percent = sand / total * 100

    if sand_percent >= 50:
        return "Sand"
    elif silt_percent >= 50:
        return "Silt"
    elif clay_percent >= 50:
        return "Clay"
    elif sand_percent >= 30:
        return "Sandy Loam"
    elif silt_percent >= 30:
        return "Silt Loam"
    elif clay_percent >= 30:
        return "Clay Loam"
    else:
        return "Loam"


soil_properties_df["soil texture"] = soil_properties_df.apply(soil_texture, axis=1)
print(soil_properties_df)


soil_properties_df.to_csv('soil_properties.csv', index = True)



In [ ]:
# Time-series point data

# set start and end date (end date will be included
# in the time period for which data is downloaded)
start_date, end_date = '2020-12-31', '2021-12-31'  # time in UTC choose start date one day before time of interest
# for position east of 0° meridian for covering all hours of interest

In [ ]:
# download era5 dataset

variable = "wefesiteanalyst"
target_file = 'era5_wefesiteanalyst_'+name+'.nc'
print(target_file)

ds = era5.get_era5_data_from_datespan_and_position(
    variable=variable,
    start_date=start_date, end_date=end_date,
    latitude=lat, longitude=lon,
    target_file=target_file)

In [ ]:
era5_wefe = pd.read_csv(r"wefe.csv")
era5_wefe.head()

In [ ]:
# plot global horizontal irradiance
import matplotlib.pyplot as plt

era5_wefe.loc[:, ['ghi']].plot(title='Irradiance')
plt.xlabel('Time')
plt.ylabel('Irradiance in $W/m^2$')